In [1]:
# pip install webdriver-manager

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import time
import math

import warnings
warnings.filterwarnings('ignore')

In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

In [4]:
# 크롬 드라이버 실행
driver = webdriver.Chrome(ChromeDriverManager().install())

In [5]:
# 카카오 맵으로 이동
url = "https://map.kakao.com/"
driver.get(url)

In [6]:
searchloc = '안동 한옥숙소'

search_area = driver.find_element(By.XPATH, r'//*[@id="search.keyword.query"]') # 카카오맵 검색창
search_area.send_keys(searchloc)  # 검색어 전달
driver.find_element(By.XPATH, r'//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)  # 돋보기

time.sleep(5)

driver.find_element(By.XPATH, r'//*[@id="info.main.options"]/li[2]/a').send_keys(Keys.ENTER)  # 장소 탭

In [7]:
room_list = []  # 숙소 정보 저장

In [8]:
def roomNamePrint():
    time.sleep(0.2)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    room_lists = soup.select('.placelist > .PlaceItem') # 장소 정보를 모두 가져옴
    for i, room in enumerate(room_lists):
        temp = []
        
        name = room.select('.head_item > .tit_name > .link_name')[0].text
        score = room.select('.rating > .score > em')[0].text
        addr = room.select('.addr > p')[0].text
        
        # 상세정보 탭으로 이동
        driver.find_element(By.XPATH, r'//*[@id="info.search.place.list"]/li['+str(i+1)+']/div[5]/div[4]/a[1]').send_keys(Keys.ENTER)
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(2)
        rev = extract_review()  # 리뷰 추출 함수 실행
        
        
        # 하나의 리스트로 만들어 room_list에 추가
        temp.append(name)
        temp.append(score)
        temp.append(addr[3:])
        temp.append(rev)
        
        room_list.append(temp)

In [9]:
def extract_review():
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 후기 목록 찾기
    review_lists = soup.select('.list_evaluation > li')
    
    count = 0
    rev = []
    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        for review in review_lists:
            comment = review.select('.txt_comment > span')[0].text  # 리뷰
            if len(comment) != 0:
                rev.append(comment)
    # 없으면 빈칸 추가
    else:
        rev.append(' ')
        
    
    # 다시 검색 탭으로 전환
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(2)
    
    return rev

In [10]:
page = 1  # 현재 페이지
page2 = 1  # 5개 중 몇번째인지(버튼이 5개씩있어서 6번째가 되면 다시 1로 바꿔줘야함)

# 전주 한옥숙소: 8페이지
# 북촌 한옥숙소: 2페이지
# 서촌 한옥숙소: 2페이지
# 서울 한옥숙소: 3페이지
# 안동 한옥숙소: 2페이지
for i in range(1, 9):
    try:
        page2 += 1
        print(page, 'page')
        
        # 페이지 버튼 번호(1에서 5 사이 값)
        if i > 5:
            xpath = '/html/body/div[5]/div[2]/div[1]/div[7]/div[6]/div/a['+str(i-5)+']'
        else:
            xpath = '/html/body/div[5]/div[2]/div[1]/div[7]/div[6]/div/a['+str(i)+']'
        
        driver.find_element(By.XPATH, xpath).send_keys(Keys.ENTER)  # 페이지 선택
        roomNamePrint()  # 숙소 정보 크롤링
        
        
        # page2가 5를 넘어가면 다시 1로 바꿔주고 다음 버튼 클릭
        if page2 > 5:
            page2 = 1
            driver.find_element(By.XPATH, r'//* [@id="info.search.page.next"]').send_keys(Keys.ENTER)
        
        page += 1
    
    except:
        break
        
print('크롤링 완료')

1 page
2 page
3 page
크롤링 완료


In [11]:
room_list

[['안동한옥펜션',
  '4.8',
  '안동시 임하면 금소길 348',
  ['이용안내에  죽실은 방이2개가 3개로되어있내요',
   '새롭게 에어콘설치, 방안 천장을 하여 아주 편한하고 시원하고 지내다가 갑니다.  주변에 많이 홍보 할께요.^^',
   '아주 좋아요  방마다 에어컨이 다있고  시골이라서 조용하고 촌에 공기도 좋고 아주좋네요']],
 ['연한옥펜션', '0.0', '안동시 서동문로 222-16', [' ']],
 ['군자마을한옥스테이', '0.0', '안동시 와룡면 군자리길 41', [' ']],
 ['율시헌한옥스테이', '5.0', '안동시 남후면 암산길 258-9', []],
 ['서우재 한옥스테이', '3.0', '안동시 어가골아랫1길 31-4', ['아주 고즈넉하고 좋아용', '이웃과 상생 하세요 !']],
 ['한옥스테이 루가', '0.0', '안동시 풍산읍 소산웃골길 11-6', [' ']],
 ['스르르 한옥스테이', '0.0', '안동시 태화3길 28', [' ']],
 ['108한옥', '0.0', '안동시 법상윗길 6', [' ']],
 ['별아래한옥펜션', '0.0', '안동시 임하면 인덕길 86', [' ']],
 ['한옥스테이 만소당게스트하우스',
  '1.0',
  '안동시 풍천면 한옥마을1길 6-9',
  ['인터넷상 정보로는 VIP가족실,  VIP비즈니스실이 각각 따로 있는 것처럼 보이는데 실상은 VIP가족실에서 방 하나(사랑방?)를 빼면 그게 VIP비즈니스실이라네요..그리고, VIP비즈니스실 기준인원이 2인(주말 30만원)이고 최대 7인까지받는다면서 초과인원 1명당 3만원을 더 달라고.. 어처구니가 없고 사기당한 느낌!  * 인터넷에 4인기준이라고 올려져 있는 걸 보여주니 그때서야 미안한 척 수긍하고.. 방 2개 싸이즈와 침구류를 실제 살펴보니 각 2명씩 4명 기본 셋팅이 맞는데..(그제까지만 해도 VIP비즈니스실이 4인기준 주말 30만원이었는데 업체문의로 바뀌어 있네요.. 기준인원 4인은 왜 그대로 두고 가

In [12]:
room_list[0]

['안동한옥펜션',
 '4.8',
 '안동시 임하면 금소길 348',
 ['이용안내에  죽실은 방이2개가 3개로되어있내요',
  '새롭게 에어콘설치, 방안 천장을 하여 아주 편한하고 시원하고 지내다가 갑니다.  주변에 많이 홍보 할께요.^^',
  '아주 좋아요  방마다 에어컨이 다있고  시골이라서 조용하고 촌에 공기도 좋고 아주좋네요']]

In [13]:
room_list[0][3] # 리뷰

['이용안내에  죽실은 방이2개가 3개로되어있내요',
 '새롭게 에어콘설치, 방안 천장을 하여 아주 편한하고 시원하고 지내다가 갑니다.  주변에 많이 홍보 할께요.^^',
 '아주 좋아요  방마다 에어컨이 다있고  시골이라서 조용하고 촌에 공기도 좋고 아주좋네요']

In [14]:
name = []
score = []
address = []
review = []
for r in room_list:
    name.append(r[0])
    score.append(r[1])
    address.append(r[2])
    review.append(r[3])

In [15]:
hanok = pd.DataFrame({'name':name, 'score':score, 'address':address, 'review':review})
hanok

,name,score,address,review
0,안동한옥펜션,4.8,안동시 임하면 금소길 348,"[이용안내에 죽실은 방이2개가 3개로되어있내요, 새롭게 에어콘설치, 방안 천장을 ..."
1,연한옥펜션,0.0,안동시 서동문로 222-16,[ ]
2,군자마을한옥스테이,0.0,안동시 와룡면 군자리길 41,[ ]
3,율시헌한옥스테이,5.0,안동시 남후면 암산길 258-9,[]
4,서우재 한옥스테이,3.0,안동시 어가골아랫1길 31-4,"[아주 고즈넉하고 좋아용, 이웃과 상생 하세요 !]"
5,한옥스테이 루가,0.0,안동시 풍산읍 소산웃골길 11-6,[ ]
6,스르르 한옥스테이,0.0,안동시 태화3길 28,[ ]
7,108한옥,0.0,안동시 법상윗길 6,[ ]
8,별아래한옥펜션,0.0,안동시 임하면 인덕길 86,[ ]
9,한옥스테이 만소당게스트하우스,1.0,안동시 풍천면 한옥마을1길 6-9,"[인터넷상 정보로는 VIP가족실, VIP비즈니스실이 각각 따로 있는 것처럼 보이는..."


In [16]:
hanok.to_csv('안동.csv', index=False, encoding="utf-8-sig")